In [1]:
import pandas as pd
import glob
import nltk
from nltk.tokenize import word_tokenize as WordTokenizer
import pickle 

In [2]:
# set the path of all files
spam_test_files = r'data\spam-test\*.txt'
not_spam_test_files=r'data\nonspam-test\*.txt'

# method to get files data from disk
def get_DatafromFiles(file_path):
    filesList = []
    file_data_list=[]
    for files in glob.glob(file_path):
        filesList.append(files) 
    for file in filesList:
        with open(file,'r') as curfile:
            file_data_list.append(curfile.readline())
    return file_data_list
spam_test_df=pd.DataFrame(get_DatafromFiles(spam_test_files),columns=['email'])
spam_test_df['actual_label']='spam'
not_spam_test_df=pd.DataFrame(get_DatafromFiles(not_spam_test_files),columns=['email'])
not_spam_test_df['actual_label']='not-spam'

test_df=not_spam_test_df.append(spam_test_df)
test_df.reset_index(drop=True,inplace=True)

In [3]:
#method to genrate tokens
def word_tokenizer(data):
    return str(nltk.word_tokenize(data))

In [4]:
# get the tokens for prediction
test_df_tokenized =test_df['email'].apply(lambda x:word_tokenizer(x))

In [5]:
#get the vectors from saved file
TfidfVectorizer_pickle_file=r'models\TfidfVectorizer_pickle.dat'
TfidfVectorizer_pickle = open(TfidfVectorizer_pickle_file, 'rb')
tfidf_vectorizer =pickle.load(TfidfVectorizer_pickle)

#transform the data
test_df_vec=tfidf_vectorizer.transform(test_df_tokenized)
TfidfVectorizer_pickle.close()

In [6]:
#get the saved classifer model for prediction
Classifier_pickle_file=r'models\Classifier_pickle.dat'
Classifier_pickle = open(Classifier_pickle_file, 'rb')
Classifier=pickle.load(Classifier_pickle)
Classifier_pickle.close()

In [7]:
#get the prediction result and confidence score
predict_test=pd.DataFrame(Classifier.predict(test_df_vec),columns=['predicted_label'])
predict_proba_test=pd.DataFrame(Classifier.predict_proba(test_df_vec))
predict_proba_test=pd.DataFrame(round(predict_proba_test.max(axis=1)*100).astype(int),columns=['confidence_score'])

In [8]:
#combine all data frames for final result
result1 = test_df.join(predict_test)
final_result=result1.join(predict_proba_test)
final_result['confidence_score']=final_result['confidence_score'].astype(str)+' %'

In [9]:
#save the result into file
final_result.to_csv('result\predicted_result.csv',index = False)